# Ejercicio final de la semana 1

Para demostrar que estás familiarizado con la API de OpenAI y también con Ollama, crea una herramienta que responda a una pregunta técnica
y la explique. ¡Esta es una herramienta que podrás usar durante el curso!

In [ ]:
# imports
# Si esto falla, verifica que esté ejecutándose desde un entorno "activado" con (llms) en el símbolo del sistema

import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI


In [ ]:
# constantes

MODEL_GPT = 'gpt-4o-mini'
MODEL_LLAMA = 'llama3.2'
messagesG=[]

In [ ]:
# Configura el entorno
# Inicialización

load_dotenv()
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key[:8]=='sk-proj-':
    print("La clave de API parece buena")
else:
    print("¿Puede haber un problema con tu clave API? ¡Visita el cuaderno de resolución de problemas!")
openai = OpenAI()


In [ ]:
def add_message(role, content, reset=False):
    """
    Agrega un mensaje al historial. Puede reiniciar la conversación si se indica.

    Args:
        role (str): 'system', 'user' o 'assistant'
        content (str): texto del mensaje
        reset (bool): si True, reinicia el historial antes de agregar
    """
    global messagesG
    if reset:
        messagesG = []
    if role not in ['system', 'user', 'assistant']:
        raise ValueError("El rol debe ser 'system', 'user' o 'assistant'")
    messagesG.append({"role": role, "content": content})

In [ ]:
def stream_output(user_prompt, system_prompt):
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=messagesG,
        stream=True
    )
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [ ]:
system_prompt="""
            Asume que eres un tutor experto en lenguaje Python, da explicaciones 
            detalladas y con ejemplos para que el alumno comprenta 
              """


In [ ]:
question = """
Explica qué hace este código y por qué:
yield from {book.get("author") for book in books if book.get("author")}
"""
add_message("system", system_prompt, reset=True)
add_message("user", question)
stream_output(user_prompt,system_prompt)